In [1]:
import requests
import uuid
from crawlerApi import crawerAPI
res = requests.get("http://www.kmseh.gov.tw/07_class/online.php")
res.close()
res.encoding = res.apparent_encoding

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, "lxml")
#print(soup.prettify())

In [12]:
table = soup.find('table',{"summary":"硬體設備"})
output_text= ""
city = "高雄市"
district = "小港區"
location = "高雄市小港區學府路115號"
organizer = "高雄社教館-教育推廣班"
figure = ""

def detailCrawler(link, class_weekday, text):
    rtntext=""
    res = requests.get(link)
    res.close()
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.text, "lxml")
    table = soup.find('table', {"bgcolor": "#F2A3B9"})
    
    #加上ID,PLACE,organizer
    ID = ""
    rtntext += city + ','+ district + ',' + location + ',' + organizer + ','
    
    for idx, row in enumerate(table.findAll('td', {"bgcolor":"#FFFFFF"})):
        if idx == 1:
            className = row.text.strip().upper()
            category = crawerAPI.courseClassify(className)
            rtntext += className + ',' + category + ','
        elif idx == 2:
            rtntext += '每周'+class_weekday+' '+row.text+','
        elif idx == 4:
            date = row.text.strip().replace(' ','').split('~')
            sdate = crawerAPI.convertDateForm(date[0],3)
            rtntext += sdate + ',' + crawerAPI.convertDateForm(date[1],3) + ','
        elif idx in (9,10):
            rtntext += '\"'+row.text+'\"'+','
        else:
            rtntext += row.text+','
    ID = ("kh_mseh"+className+sdate)
    rtntext = ID + ',' + rtntext
    rtntext += text
    return rtntext

for row in table.findAll("tr"):
    text= ""
    rowdata = row.findAll("td")
    href = "http://www.kmseh.gov.tw/07_class/"
    class_weekday = ""
    if len(rowdata) == 10:
        for idx, i in enumerate(rowdata):
            if idx == 2:
                detail_href = href + i.find('a').get('href')
            elif idx == 3:
                class_weekday = i.text.strip()
            elif idx == 8:
                singup_list_href = href + i.find('a').get('href')
            elif idx == 9:
                try:
                    singup_href = href + i.find('a').get('href')
                except:
                    singup_href = ""
        text += detail_href+"," 
        text += singup_href+","
        text += figure+","
        text += singup_list_href+"\n"
        output_text += detailCrawler(detail_href, class_weekday ,text)
print(output_text)  

kh_mseh水墨畫A(日)2016-05-27,高雄市,小港區,高雄市小港區學府路115號,高雄社教館-教育推廣班,1,水墨畫A(日),美術繪畫,每周五 09:00~11:00,5樓研習教室,2016-05-27,2016-08-26,黃榮德,1800,25人,17人,"基礎水墨運筆方式：山水畫、樹、石、雲、水、花鳥分段解說示範說明，落款用印及構圖原理之講解。","1.適合一般初學者；2.教材自備或第一堂課登記團購。",http://www.kmseh.gov.tw/07_class/online_detail.php?bull_id=9118,http://www.kmseh.gov.tw/07_class/online_register.php?bull_id=9118,,http://www.kmseh.gov.tw/07_class/online_number.php?bull_id=9118
kh_mseh水墨畫B(夜)2016-05-27,高雄市,小港區,高雄市小港區學府路115號,高雄社教館-教育推廣班,2,水墨畫B(夜),美術繪畫,每周五 19:00~21:00,5樓研習教室,2016-05-27,2016-08-26,黃榮德,1800,25人,21人,"基礎水墨運筆方式：山水畫、樹、石、雲、水、花鳥分段解說示範說明，落款用印及構圖原理之講解。","1.適合一般初學者；2.教材自備或第一堂課登記團購。",http://www.kmseh.gov.tw/07_class/online_detail.php?bull_id=9119,http://www.kmseh.gov.tw/07_class/online_register.php?bull_id=9119,,http://www.kmseh.gov.tw/07_class/online_number.php?bull_id=9119
kh_mseh水墨畫進階(日)2016-05-25,高雄市,小港區,高雄市小港區學府路115號,高雄社教館-教育推廣班,3,水墨畫進階(日),美術繪畫,每周三 09:00~11:00,5樓研習教室,2016-05-25,2016-08-17,黃榮德,1800,25人,21人,"山水花鳥畫之各種樹、石、雲、水及花鳥之構圖原理與組合，透視觀念及創作之要領和特殊技法之講解

In [10]:
with open('./output_file/'+'kh_mseh.csv','w', encoding ='utf8') as outputfile:
    #title
    outputfile.write('ID,city,district,location,organizer,serial_number,course_name,category,classtime,classroom,start_date,end_date,teacher,charge,num_of_people,num_of_applied,intro,note,relate_website,signup,figure,apply_list'+'\n')
    outputfile.write(output_text)

In [11]:
crawerAPI.csvtojson('kh_mseh.csv','kh_mseh.json')